In [1]:
# gensim word2vec实例

In [2]:
'''
gensim word2vec API概述
在gensim中，word2vec相关的API都在包gensim.models.word2vec中。
和算法有关的参数都在类gensim.models.word2vec.Word2Vec中。

算法需要注意的参数有：
1.sentences 我们需要分析的语料，可以是一个列表，或者从文件中遍历读出。
2.size 词向量的维度，默认值是100。
    这个维度的取值一般与我们的语料的大小相关，如果是不大的语料，比如小于100M的文本语料，
    则使用默认值一般就可以。
    如果是超大的语料，建议增大维度。
3.window 即词向量上下文最大距离，这个参数在我们的算法原理篇中标记为c，
    window越大，则和某一词较远的词也会产生上下文关系。
    默认值为5.
    在实际使用中，可以根据实际的需求来动态调整这个window的大小。
    如果是小语料则这个值可以设的更小。
    对于一般的语料这个值推荐在[5,10]之间。
4.sg 即我们的word2vec两个模型的选择。
    如果是0，则是CBOW模型；
    如果是1，则是Skip-Gram模型。
    默认是0，即CBOW模型。
5.hs 即我们的word2vec两个解法的选择。
    如果是0，则是Negative Sampling;
    如果是1，则是Hierarchical Softmax.
    默认是0即 Negative Sampling.
6.negative 即使用Negative Sampling时负采样的个数，默认是5.
    推荐在[3,10]之间。
    这个参数在我们的算法原理篇中标记为ng。
7.cbow_mean 仅用于CBOW在做投影的时候，
    为0，则算法中的x_{w}为上下文的词向量之和；
    为1，则为上下文的词向量的平均值。
    默认值为1，不推荐修改默认值。
8.min_count 需要计算词向量的最小词频。
    这个值可以去掉一些很生僻的低频词，默认是5.
    如果是小语料，可以调低这个值。
9.iter 随机梯度下降法中迭代的最大次数，默认是5.
    对于大语料，可以增大这个值。
10.alpha 在随机梯度下降法中迭代的初始步长。
    算法原理篇中标记为η，默认是0.025.
11.min_alpha 由于算法支持在迭代的过程中逐渐减小步长,
    此处为最小的迭代步长值。
    随机梯度下降中每轮的迭代步长可以由iter,alpha,min_alpha一起得出。
    对于大语料，需要对alpha,min_alpha,iter一起调参，来选择合适的三个值。
'''

In [23]:
# 《人民的名义》的小说原文作为语料。

In [24]:
import os
import jieba
import jieba.analyse
from gensim.models import word2vec

In [25]:
FILENAME = 'in_the_name_of_people.txt'

In [28]:
# 调整字典，添加词汇
jieba.suggest_freq('沙瑞金', True)
jieba.suggest_freq('田国富', True)
jieba.suggest_freq('高育良', True)
jieba.suggest_freq('侯亮平', True)
jieba.suggest_freq('钟小艾', True)
jieba.suggest_freq('陈岩石', True)
jieba.suggest_freq('欧阳菁', True)
jieba.suggest_freq('易学习', True)
jieba.suggest_freq('王大路', True)
jieba.suggest_freq('蔡成功', True)
jieba.suggest_freq('孙连城', True)
jieba.suggest_freq('季昌明', True)
jieba.suggest_freq('丁义珍', True)
jieba.suggest_freq('郑西坡', True)
jieba.suggest_freq('赵东来', True)
jieba.suggest_freq('高小琴', True)
jieba.suggest_freq('赵瑞龙', True)
jieba.suggest_freq('林华华', True)
jieba.suggest_freq('陆亦可', True)
jieba.suggest_freq('刘新建', True)
jieba.suggest_freq('刘庆祝', True)

1

In [27]:
# 分词
with open(FILENAME,'r') as f:
    document = f.read()
    document_cut = jieba.cut(document)
    result = ' '.join(document_cut)

with open('people_segments.txt','w') as f:
    f.write(result)

In [22]:
# 加载停用词，由于此处是使用word2vec，考虑到上下文，所以不加载停用词
# stpwrd = open('stop_words.txt','r').read()
# stpwrdlst = stpwrd.splitlines()

In [29]:
# 用word2vec提供的LineSentence类来读文件。
sentences = word2vec.LineSentence('people_segments.txt')

In [33]:
# 构造Word2Vec实例
model = word2vec.Word2Vec(sentences, hs=1, min_count =1, window =3, size=100)

In [34]:
# 常用应用介绍：

In [36]:
# 1.找出某一个词向量最相近的词集合
print(model.wv.similar_by_word('沙瑞金',topn=10))

[('高育良', 0.9401112794876099), ('李达康', 0.9266355037689209), ('易学习', 0.9027300477027893), ('肖钢玉', 0.8982406854629517), ('报告', 0.8944628238677979), ('祁同伟', 0.8936658501625061), ('一下', 0.8913800716400146), ('陈海', 0.8909031748771667), ('介绍', 0.8893004655838013), ('又', 0.8892979621887207)]


In [38]:
# 2.看两个词向量的相近程度
print(model.wv.similarity('沙瑞金','高育良'))
print(model.wv.similarity('李达康','王大路'))

0.9401112877580539
0.8929893437592946


In [42]:
# 3.找出不同类的词
print(model.wv.doesnt_match(("沙瑞金","高育良","李达康","刘庆祝",'贪污')))

贪污
